In [1]:
from datetime import datetime
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY
from client import FtxClient

ModuleNotFoundError: No module named 'ciso8601'

In [11]:
url = 'https://ftx.com/api/markets'

#get BTCUSD spot price
base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']

#define resolutions
hourly = str(60*60)
daily = str(60*60*24)


In [33]:
#define function to fetch multiple months of data for 2021
def get_historical2021(m, d, n_months):
    """ grabs multiple months of data by sending multiple queries to FTX API
    
    Parameters:
        m(int): month of year
        d(int): day of week
        n_months(int): number of months from starting date 
                        (ex. input n_months = 3 if you want 3 months of data beginning from start date)
        
    Returns:
        df(pd.DataFrame): pandas dataframe containing all data for specified number of months since start date 
    
    """
    while n_months > 0:
        #fetch data for the first month beginning with the start date 
        if m == m:
            
            start_date = datetime(2021, m, d).timestamp()
            end_date = datetime(2021, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            df = pd.DataFrame(historical['result'])
                        
            #add 1 to starting month
            m += m
            
        
        #fetch data for the next month
        else:
            start_date = datetime(y, m, d).timestamp()
            end_date = datetime(y, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            #convert json to pd df and append all rows except for the first repeating row to df
            next_month = pd.DataFrame(historical['result'])
            df.append(next_month.iloc[1: , :])
            
            m += m
            
        #subtract n_months counter
        n_months -= n_months
    else:
        return df

#runs function and saves return df to df_2021
df_2021 = get_historical2021(1,1,3)  

#convert to datetime and drop unnecessary rows df_2021
df_2021['date'] = pd.to_datetime(df_2021['time']/1000, unit='s', origin='unix')

df_2021.drop(['startTime', 'time'], axis=1, inplace=True)

df_2021        


,open,high,low,close,volume,date
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00
...,...,...,...,...,...,...
740,32575.0,33563.0,32511.0,33448.0,9.540823e+06,2021-02-01 01:00:00
741,33448.0,33858.0,33243.0,33672.0,6.190308e+06,2021-02-01 02:00:00
742,33672.0,33779.0,33488.0,33592.0,3.189237e+06,2021-02-01 03:00:00
743,33592.0,33953.0,33454.0,33578.0,4.414067e+06,2021-02-01 04:00:00


In [ ]:
def get_historical2022(m, d, n_months):
    while n_months > 0:
        #fetch data for the first month beginning with the start date 
        if m == m:
            
            start_date = datetime(2022, m, d).timestamp()
            end_date = datetime(2022, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            df = pd.DataFrame(historical['result'])
                        
            #add 1 to starting month
            m += m
        
        #fetch data for the next month
        else:
            start_date = datetime(y, m, d).timestamp()
            end_date = datetime(y, m+1, d).timestamp()
            
            historical = requests.get(
                f'{request_url}/candles?resolution={hourly}&start_time={start_date}&end_time={end_date}'
                ).json()
            
            #convert json to pd df and append all rows except for the first repeating row to df
            next_month = pd.DataFrame(historical['result'])
            df.append(next_month.iloc[1: , :])
            
            
            #add 1 to starting month
            m += m
        #subtract n_months counter
        n_months -= n_months
    else:
        return df

get_historical2021(1,1,4)           
            